# 0_create_US_stocks_data

### source
- etf list : etf db
- stocks list: fdr
- prices: yahoo finanace

## Imports & Settings

In [52]:
import pandas as pd
import yfinance as yf
import FinanceDataReader as fdr
from pathlib import Path
from tqdm import tqdm
from tools.usetf_list import USETF_LIST
import warnings
warnings.filterwarnings('ignore')

In [53]:
def download_prices(ticker_list:list, src = 'yfinance', start_date="2000-01-01", end_date="2024-07-28"):
    prices_list = []
    for ticker in tqdm(ticker_list):
        if src == 'yfinance':
            stock = yf.Ticker(ticker)
            price_info = stock.history(start=start_date, end=end_date)
        elif src == 'fdr':
            price_info = fdr.DataReader(ticker, start=start_date, end=end_date)
        price_info['ticker'] = ticker
        # price_info.reset_index(inplace=True)
        prices_list.append(price_info)
    prices_df = pd.concat(prices_list)
    return prices_df

def remake_df(prices_df):
    # 날짜 형식으로 인덱스 변환 시도
    if pd.api.types.is_datetime64_any_dtype(prices_df.index):
        prices_df.index = prices_df.index.strftime('%Y-%m-%d')
    
    # 불필요한 컬럼 제거
    columns_to_drop = ['adj close', 'capital gains', 'stock splits']
    for column in columns_to_drop:
        if column in prices_df.columns:
            prices_df = prices_df.drop([column], axis=1)
    
    # 인덱스 재설정 및 컬럼명 정리
    prices_df = prices_df.reset_index()
    prices_df.columns = prices_df.columns.str.lower()
    prices_df.columns = prices_df.columns.str.replace("dividends", "dividend")
    
    # 인덱스 설정 및 중복 제거
    if 'ticker' in prices_df.columns and 'date' in prices_df.columns:
        prices_df = prices_df.set_index(['ticker', 'date'])
        prices_df = prices_df[~prices_df.index.duplicated(keep='first')]
    
    return prices_df
    
# def remake_df(prices_df):

#     try:
#         prices_df.index = prices_df.index.strftime('%Y-%m-%d')
#     except:
#         pass    
#     try: 
#         prices_df = prices_df.drop(['adj close'], axis=1)
#     except:
#         pass
#     try: 
#         prices_df = prices_df.drop(['capital gains'], axis=1)
#     except:
#         pass
#     try: 
#         prices_df = prices_df.drop(['stock splits'], axis=1)
#     except:
#         pass
    
#     prices_df = prices_df.reset_index()
#     prices_df.columns = prices_df.columns.str.lower()
#     prices_df.columns = prices_df.columns.str.replace("dividends", "dividend")
#     prices_df = prices_df.set_index(['ticker', 'date'])
#     prices_df = prices_df[~prices_df.index.duplicated(keep='first')]
#     return prices_df

## Set Data Store path

In [30]:
DATA_STORE = Path('E:/mlft/data/assets.h5')
print(DATA_STORE.cwd())

E:\mlft\data


In [45]:
start_date="2000-01-01"
end_date="2024-07-28"

## Download US ETF

### download_usetf list from etfdb

In [5]:
key = f'us/etfs/tickers/etfdb/'

In [6]:
etf_list = USETF_LIST()
df = etf_list.get_etf_list()
df = df.set_index('symbol')
df.to_hdf(DATA_STORE, key, format='t')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 136/136 [27:58<00:00, 12.34s/it]


### download_usetf_prices from yfinance

In [32]:
key = f'us/etfs/prices/yfinance'

In [14]:
df = download_prices(df.index, start_date=start_date, end_date=end_date)
df = remake_df(df)
df.to_hdf(DATA_STORE, key, format='t')
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6190247 entries, ('SPY', '2000-01-03') to ('RBUF', '2024-07-26')
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   open           float64
 1   high           float64
 2   low            float64
 3   close          float64
 4   volume         float64
 5   dividend       float64
 6   stock splits   float64
 7   capital gains  float64
dtypes: float64(8)
memory usage: 401.8+ MB


## Download US stocks

### download_snp500_stocks list from fdrc

In [ ]:
key = f'us/snp500/tickers/fdr'

In [28]:
stock_list = fdr.StockListing('S&P500')
stock_list.columns=stock_list.columns.str.lower()
df = stock_list.set_index('symbol')
df.to_hdf(DATA_STORE, key, format='t')
df.head()

### download_snp500_prices from yfinance

In [ ]:
key = f'us/snp500/prices/yfinance'

In [29]:
df = download_prices(df.index, start_date=start_date, end_date=end_date)
df = remake_df(df)
df.to_hdf(DATA_STORE, key, format='t')
df.info()

$BRKB: possibly delisted; No timezone found
Failed to get ticker 'BFB' reason: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))
$BFB: possibly delisted; No timezone found


### download_nasdaq_stocks list from fdr

In [41]:
key = f'us/nasdaq/tickers/fdr'

In [42]:
stock_list = fdr.StockListing('NASDAQ')
stock_list.columns=stock_list.columns.str.lower()
df = stock_list.set_index('symbol')
df.to_hdf(DATA_STORE, key, format='t')
df.head()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3737/3737 [00:10<00:00, 343.05it/s]


,name,industrycode,industry
symbol,,,
AAPL,Apple Inc,57106020,전화 및 소형 장치
MSFT,Microsoft Corp,57201020,소프트웨어
NVDA,NVIDIA Corp,57101010,반도체
AMZN,Amazon.com Inc,53402010,백화점
META,Meta Platforms Inc,57201030,온라인 서비스


### download_nasdaq_prices from yfinancec

In [43]:
key = f'us/nasdaq/prices/yfinance'

In [ ]:
df = download_prices(df.index, start_date=start_date, end_date=end_date)
df = remake_df(df)
df.to_hdf(DATA_STORE, key, format='t')
df.info()

  7%|████████▏                                                                                                             | 261/3737 [02:21<5:33:10,  5.75s/it]Failed to get ticker 'CGNX' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)
$CGNX: possibly delisted; No timezone found
  7%|████████▎                                                                                                             | 262/3737 [02:33<7:14:37,  7.50s/it]Failed to get ticker 'ROKU' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)
$ROKU: possibly delisted; No timezone found
  7%|████████▎                                                                                                             | 263/3737 [02:45<8:42:57,  9.03s/it]Failed to get ticker 'ITCI' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)
$ITCI: possibly delisted; No t

### download_nyse_stocks list from fdr

In [ ]:
key = f'us/nyse/tickers/fdr'

In [ ]:
stock_list = fdr.StockListing('NYSE')
stock_list.columns=stock_list.columns.str.lower()
df = stock_list.set_index('symbol')
df.to_hdf(DATA_STORE, key, format='t')
df.head()

### download_nyse_prices from yfinancec

In [ ]:
key = f'us/nyse/prices/yfinance'

In [ ]:
df = download_prices(df.index, start_date=start_date, end_date=end_date)
df = remake_df(df)
df.to_hdf(DATA_STORE, key, format='t')
df.info()

## Download KR stocks

### download_etf list from fdr

In [21]:
key = f'kr/etf/tickers/fdr'

In [23]:
stock_list = fdr.StockListing('ETF/KR')
stock_list.columns=stock_list.columns.str.lower()
df = stock_list.set_index('symbol')
df.to_hdf(DATA_STORE, key, format='t')
df.head()

,category,name,price,risefall,change,changerate,nav,earningrate,volume,amount,marcap
symbol,,,,,,,,,,,
459580,6,KODEX CD금리액티브(합성),1043380,2,70,0.01,1043379.0,0.8833,533604,556750,94960
357870,6,TIGER CD금리투자KIS(합성),54685,3,0,0.00,54687.0,0.8762,189030,10337,66364
069500,1,KODEX 200,38080,2,450,1.20,38145.0,4.0508,2321332,88405,62032
423160,6,KODEX KOFR금리액티브(합성),107540,2,15,0.01,107529.0,0.8819,99116,10659,45217
360750,4,TIGER 미국S&P500,18945,2,180,0.96,18845.0,7.5666,2878170,54491,42266


### download_etf_prices from fdr

In [24]:
key = f'kr/etf/prices/fdr'

In [25]:
df = download_prices(df.index, src ='fdr', start_date=start_date, end_date=end_date)
df = remake_df(df)
df.to_hdf(DATA_STORE, key, format='t')
df.head()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 873/873 [08:23<00:00,  1.73it/s]


open     high      low    close  volume    change
ticker date                                                            
459580 2023-06-08  1000970  1000970  1000960  1000965    6877       NaN
       2023-06-09  1000965  1001245  1000955  1001245    8924  0.000280
       2023-06-12  1001400  1001400  1001390  1001400    6657  0.000155
       2023-06-13  1001445  1001445  1001435  1001445    5902  0.000045
       2023-06-14  1001545  1001545  1001530  1001545    8275  0.000100

### download_kospi_stocks list from fdr

In [ ]:
key = f'kr/kospi/tickers/fdr'

In [ ]:
stock_list = fdr.StockListing('KOSPI')
stock_list.columns=stock_list.columns.str.lower()
df = stock_list.set_index('symbol')
df.to_hdf(DATA_STORE, key, format='t')
df.head()

### download_kospi_prices from fdr

In [ ]:
key = f'kr/kospi/prices/naver'

In [ ]:
df = download_prices(df.index, src ='fdr', start_date=start_date, end_date=end_date)
df = remake_df(df)
df.to_hdf(DATA_STORE, key, format='t')
df.head()

## Download CHN stocks

### download_shaghai_stocks list from fdr

In [ ]:
key = f'chn/sse/tickers/fdr'

In [ ]:
stock_list = fdr.StockListing('SSE')
stock_list.columns=stock_list.columns.str.lower()
df = stock_list.set_index('symbol')
df.to_hdf(DATA_STORE, key, format='t')
df.head()

### download_shaghai_prices from fdr

In [ ]:
key = f'chn/sse/prices/fdr'

In [ ]:
df = download_prices(df.index, src ='fdr', start_date=start_date, end_date=end_date)
df = remake_df(df)
df.to_hdf(DATA_STORE, key, format='t')
df.head()

### download_shenzen_stocks list from fdrc

In [ ]:
key = f'chn/szse/tickers/fdr'

In [ ]:
stock_list = fdr.StockListing('SZSE')
stock_list.columns=stock_list.columns.str.lower()
df = stock_list.set_index('symbol')
df.to_hdf(DATA_STORE, key, format='t')
df.head()

### download_shenzen_prices list from fdrc

In [ ]:
key = f'chn/szse/prices/fdr'

In [ ]:
df = download_prices(df.index, src ='fdr', start_date=start_date, end_date=end_date)
df = remake_df(df)
df.to_hdf(DATA_STORE, key, format='t')
df.head()

## Download JPN stocks

### download_tse_stocks list from fdr

In [ ]:
key = f'jpn/tse/tickers/fdr'

In [ ]:
stock_list = fdr.StockListing('TSE')
stock_list.columns=stock_list.columns.str.lower()
df = stock_list.set_index('symbol')
df.to_hdf(DATA_STORE, key, format='t')
df.head()

### download_tse_stocks prices from fdr

In [ ]:
key = f'jpn/tse/prices/fdr'

In [ ]:
df = download_prices(df.index, src ='fdr', start_date=start_date, end_date=end_date)
df = remake_df(df)
df.to_hdf(DATA_STORE, key, format='t')
df.head()

## commodity 데이터

In [ ]:
stock_list = {'HG=F':'구리', 'PA=F':'팔라듐',
        'CL=F':'크루드 오일', 'BZ=F':'브렌트유','HO=F':'등유',
        'NG=F':'천연가스',
        'GC=F':'금','SI=F':'은', 'PL=F':'백금',
        'ZC=F':'옥수수','ZO=F':'귀리', 'KE=F':'밀', 'ZM=F':'콩','ZL=F':'콩 오일','ZR=F':'쌀',
        'GF=F':'사료', 'HE=F':'돼지 고기', 'LE=F':'소고기',
        'LBS=F':'목재',
        'CC=F':'코코아', 'KC=F':'커피', 'CT=F':'원면', 'SB=F':'설탕','OJ=F':'오렌지'}
stock_list.


### download_tse_stocks prices from fdr

In [ ]:
key = f'global/commodity/prices/yfinanae'

In [ ]:
df = download_prices(df.index, src ='yfinance', start_date=start_date, end_date=end_date)
df = remake_df(df)
df.to_hdf(DATA_STORE, key, format='t')
df.head()

## Money market 데이터


- St. Louis Adjusted Monetary Base
- Money Stock: M1, M2
- Velocity of Money Stock: M1, M2
- Effective Federal Funds Rate
- Treasury: 3개월, 2년, 5년, 10년, 30년
- Breakeven Inflation Rate: 5년, 10년
- Forward Inflation Expectation Rate: 5년
- TED Spread
- Bank Prime Loan Rate

In [ ]:
tags = {'BASE':'St. Louis Adjusted Monetary Base',
        'M1': 'M1 Money Stock',
        'M2':	'M2 Money Stock',
        'M1V' : 'Velocity of M1 Money Stock',
        'M2V' : 'Velocity of M2 Money Stock'}

tags = {'DFF' : 'Effective Federal Funds Rate',
        'DTB3' : '3-Month Treasury Bill: Secondary Market Rate',
        'DGS2' : '2-Year Treasury Constant Maturity Rate',
        'DGS5' : '5-Year Treasury Constant Maturity Rate',
        'DGS10' : '10-Year Treasury Constant Maturity Rate',
        'DGS30' : '30-Year Treasury Constant Maturity Rate',
        'T5YIE' : '5-year Breakeven Inflation Rate',
        'T10YIE' : '10-year Breakeven Inflation Rate',
        'T5YIFR' : '5-Year, 5-Year Forward Inflation Expectation Rate',
        'TEDRATE' : 'TED Spread',
        'DPRIME' : 'Bank Prime Loan Rate'}

tags = { 'DEXCHUS':'위안화', 'DEXJPUS':'엔화', 'DEXKOUS':'원화','DEXBZUS':'헤알화', 'DEXEUUS': '유로화'}

## Prices and Inflation


- CPI, Consumer Price Index
- Gross Domestic Product: Implicit Price Deflator

- Gross Domestic Product
- GDP Difference = Real Gross Domestic Produc - Real Potential Gross Domestic Product

- Civilian Unemployment Rate
- Natural Rate of Unemployment
- Civilian Labor Force Participation Rate
- Civilian Employment-Population Ratio
- Unemployed
- All Employees
- Initial Claims

-  Real Median Household Income in the United States (미국의 실질 중위 가구 소득)
- Real Disposable Personal Income (실질가처분소득)
- Personal Consumption Expenditures (개인소비지출)
- Personal Consumption Expenditures: Durable Goods (개인 소비 지출: 내구재)
- Personal Saving Rate (개인 저축률)
- Real Retail and Food Services Sales (실제 소매 및 식품 서비스 판매)
- Disposable personal income (가처분소득)

-  Industrial Production Index (산업생산지수)
- Capacity Utilization: Total Industry (CAPA 활용: 총 산업)
- Housing Starts: Total: New Privately Owned Housing Units Started (주택 착공: 총합: 신규 개인 주택 )
- Gross Private Domestic Investment (민간투자총액)
- Corporate Profits After Tax (법인세 후 이익(IVA 및 CCAdj 미포함)
- St. Louis Fed Financial Stress Index (세인트루이스 연방 재정압박 지수)
- Crude Oil Prices: West Texas Intermediate (WTI) - Cushing, Oklahoma (원유 가격: 서부 텍사스 중질유(WTI) - 오클라호마 쿠싱)
- Leading Index for the United States (미국의 선행 지수)
- Trade Weighted U.S. Dollar Index: Major Currencies (무역 비중 미국 달러 지수: 주요 통화)
- Trade Weighted U.S. Dollar Index: Broad (무역 비중 미국 달러 지수: 광범위)


                                                      -  Federal Debt: Total Public Debt (연방 부채: 총 공채)
- Federal Debt: Total Public Debt as Percent of Gross Domestic Product (연방 부채: 국내총생산의 비율에 따른 총 공공부채)
- Excess Reserves of Depository Institutions (초과 지급 준비금)
- Commercial and Industrial Loans, All Commercial Banks (상업 및 산업 대출, 모든 시중 은행)

In [ ]:
tags = {'CPIAUCSL': 'Consumer Price Index for All Urban Consumers: All Items',
        'CPILFESL': 'Consumer Price Index for All Urban Consumers: All Items Less Food & Energy',
        'GDPDEF': 'Gross Domestic Product: Implicit Price Deflator'}

tags = {'GDP': 'Gross Domestic Product',
        'GDPC1' : 'Real Gross Domestic Product',
        'GDPPOT': 'Real Potential Gross Domestic Product'}

tags = {'UNRATE' : 'Civilian Unemployment Rate',
        'NROU': 'Natural Rate of Unemployment (Long-Term)',
        'NROUST' : 'Natural Rate of Unemployment (Short-Term)',
        'CIVPART' : 'Civilian Labor Force Participation Rate',
        'EMRATIO' : 'Civilian Employment-Population Ratio',
        'UNEMPLOY' : 'Unemployed',
        'PAYEMS' : 'All Employees: Total nonfarm',
        'MANEMP' : 'All Employees: Manufacturing',
        'ICSA' : 'Initial Claims',
        'IC4WSA': '4-Week Moving Average of Initial Claims'}


tags = {'MEHOINUSA672N' : 'Real Median Household Income in the United States',
        'DSPIC96' : 'Real Disposable Personal Income',
        'PCE' : 'Personal Consumption Expenditures',
        'PCEDG' : 'Personal Consumption Expenditures: Durable Goods',
        'PSAVERT' : 'Personal Saving Rate',
        'RRSFS' : 'Real Retail and Food Services Sales',
        'DSPI' : 'Disposable personal income'}

tags = {'INDPRO' : 'Industrial Production Index',
        'TCU' : 'Capacity Utilization: Total Industry',
        'HOUST' : 'Housing Starts: Total: New Privately Owned Housing Units Started',
        'GPDI' : 'Gross Private Domestic Investment',
        'CP': 'Corporate Profits After Tax (without IVA and CCAdj)',
        'STLFSI' : 'St. Louis Fed Financial Stress Index',
        'DCOILWTICO' : 'Crude Oil Prices: West Texas Intermediate (WTI) - Cushing, Oklahoma',
        'USSLIND': 'Leading Index for the United States',
        'DTWEXM' : 'Trade Weighted U.S. Dollar Index: Major Currencies',
        'DTWEXB' : 'Trade Weighted U.S. Dollar Index: Broad'}


tags = {'GFDEBTN' : 'Federal Debt: Total Public Debt',
        'GFDEGDQ188S' : 'Federal Debt: Total Public Debt as Percent of Gross Domestic Product',
        'EXCSRESNW' : 'Excess Reserves of Depository Institutions',
        'TOTCI' : 'Commercial and Industrial Loans, All Commercial Banks'}

## check data in HDF5 format

In [8]:
with pd.HDFStore(DATA_STORE, mode='r') as store:
    all_keys = store.keys()       
all_keys

['/engineered_features',
 '/us/snp500/tickers/fdr',
 '/us/snp500/prices/yfinance',
 '/us/etfs/tickers/etfdb',
 '/us/etfs/prices/yfinance',
 '/stooq/us/nysemkt/stocks/prices',
 '/stooq/us/nysemkt/stocks/tickers',
 '/stooq/us/nyse/stocks/prices',
 '/stooq/us/nyse/stocks/tickers',
 '/stooq/us/nyse/etfs/prices',
 '/stooq/us/nyse/etfs/tickers',
 '/stooq/us/nasdaq/stocks/prices',
 '/stooq/us/nasdaq/stocks/tickers',
 '/stooq/us/nasdaq/etfs/prices',
 '/stooq/us/nasdaq/etfs/tickers',
 '/stooq/jp/tse/stocks/prices',
 '/stooq/jp/tse/stocks/tickers',
 '/sp500/fred',
 '/sp500/stocks',
 '/sp500/stooq',
 '/quandl/wiki/prices',
 '/quandl/wiki/stocks',
 '/fred/assets']

In [54]:
key = f'us/etfs/prices/yfinance'

In [55]:
with pd.HDFStore(DATA_STORE, mode='r') as store:
    df = store[key]  

In [56]:
df

open       high        low      close      volume  \
ticker date                                                                 
SPY    2000-01-03  95.094217  95.094217  92.287896  93.290154   8164300.0   
       2000-01-04  92.067417  92.408185  89.571794  89.641953   8089800.0   
       2000-01-05  89.762247  90.784551  88.038363  89.802338  12177900.0   
       2000-01-06  89.561756  90.764466  88.359047  88.359047   6227200.0   
       2000-01-07  90.002758  93.490616  89.842397  93.490616   8066500.0   
...                      ...        ...        ...        ...         ...   
RBUF   2024-07-22  25.549999  25.639999  25.549999  25.636999      2300.0   
       2024-07-23  25.693001  25.693001  25.680000  25.680000       600.0   
       2024-07-24  25.639999  25.639999  25.565001  25.565001      4600.0   
       2024-07-25  25.629999  25.670000  25.620001  25.620001      8500.0   
       2024-07-26  25.629999  25.719999  25.629999  25.684999      1000.0   

                   dividend  
ticker date                  
SPY    2000-01-03       0.0  
       2000-01-04       0.0  
       2000-01-05       0.0  
       2000-01-06       0.0  
       2000-01-07       0.0  
...                     ...  
RBUF   2024-07-22       0.0  
       2024-07-23       0.0  
       2024-07-24       0.0  
       2024-07-25       0.0  
       2024-07-26       0.0  

[6190247 rows x 6 columns]

---